<h2><center>IBM Employee Attrition ML Project</center></h2>
<h2><center> Feature Engineering & Model Training</center></h2>
<h4><center>Author: Akshay Pandurang Paunikar</center></h4>

In [46]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [47]:
# set working directory
import io
%cd "dataset/"

[WinError 2] The system cannot find the file specified: 'dataset/'
e:\iNeuron\Projects\IBM_Employee_Attrition\notebook\dataset


In [48]:
# Import the dataset
data = pd.read_csv("IBM_HR_DATA.csv")

In [49]:
# check the first five records
data = data.drop(["Unnamed: 0"], axis=1)
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,Female,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,Male,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,4,Male,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,Female,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,Male,...,3,4,1,6,3,3,2,2,2,2


In [50]:
# shape of the data
data.shape

(1470, 31)

In [51]:
# create an instance of Label Encoder, One Hot Encode, Standard Scaler
le = LabelEncoder()
one_hot = OneHotEncoder()
ss = StandardScaler()

In [52]:
# split data into independent features and target variable
X = data.drop(['Attrition'], axis=1)
y = data['Attrition']

In [53]:
# numerical features and categorical features
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

In [54]:
# create pipelines for categorical and numerical data
num_pipeline = Pipeline(
    steps=[
        ("Scaler", StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ("one hot", OneHotEncoder()),
        ("Scaler", StandardScaler(with_mean=False))
    ]
)

In [55]:
# creating preprocessor object
preprocessor = ColumnTransformer([
    ("num_pipeline", num_pipeline, num_features),
    ("cat_pipeline", cat_pipeline, cat_features)
])

In [56]:
# applying preprocesing object to features
X = preprocessor.fit_transform(X)

In [57]:
# # divide the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (1029, 51)
y_train: (1029,)
X_test: (441, 51)
y_test: (441,)
